In [2]:
from blazingsql import BlazingContext
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask_cuda import initialize

In [3]:
enable_tcp_over_ucx = True
enable_nvlink = False
enable_infiniband = False

In [4]:
import os
os.environ["UCX_LOG_LEVEL"] = "TRACE"

In [5]:
initialize.initialize(create_cuda_context=True,
                      enable_tcp_over_ucx=enable_tcp_over_ucx,
                      enable_nvlink=enable_nvlink,
                      enable_infiniband=enable_infiniband)
cluster = LocalCUDACluster(protocol="ucx",
                           enable_tcp_over_ucx=enable_tcp_over_ucx,
                           enable_nvlink=enable_nvlink,
                           enable_infiniband=enable_infiniband)

client = Client(cluster)
client

Client Scheduler: ucx://127.0.0.1:39001 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 50.39 GB


In [6]:
!ifconfig

cscotun0: flags=4305<UP,POINTOPOINT,RUNNING,NOARP,MULTICAST>  mtu 1406
        inet 10.2.167.188  netmask 255.255.240.0  destination 10.2.167.188
        inet6 fe80::49e9:cde6:a28d:ee99  prefixlen 64  scopeid 0x20<link>
        unspec 00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00  txqueuelen 500  (UNSPEC)
        RX packets 788042  bytes 739765483 (705.4 MiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 609922  bytes 128185359 (122.2 MiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

enp1s0f0u9c4i2: flags=4099<UP,BROADCAST,MULTICAST>  mtu 1500
        ether 9a:00:c6:5e:cc:d2  txqueuelen 1000  (Ethernet)
        RX packets 0  bytes 0 (0.0 B)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 0  bytes 0 (0.0 B)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

enp5s0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 192.168.50.250  netmask 255.255.255.0  broadcast 192.168.50.255
     

In [7]:
bc = BlazingContext(network_interface="enp5s0", dask_client=client, config_options={"MAX_JOIN_SCATTER_MEM_OVERHEAD": 1})

starting polling thread
starting listeners
started listeners
blazing_log
BlazingContext ready


In [8]:
import cudf
import numpy as np
df = cudf.DataFrame({chr(x): cudf.Series(np.arange(4172, dtype="float64")) for x in range(65, 66)})

import dask_cudf

ddf = dask_cudf.from_cudf(df, npartitions=2)

In [9]:
!nvidia-smi

Wed Jul 15 19:12:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro GV100        Off  | 00000000:42:00.0 Off |                  Off |
| 37%   50C    P2    45W / 250W |   1324MiB / 32508MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Quadro GV100        Off  | 00000000:43:00.0  On |                  Off |
| 45%   59C    P2    52W / 250W |   1493MiB / 32505MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
        

In [10]:
ddf = ddf.persist()

In [11]:
bc.create_table("test", ddf)
bc.create_table("test2", ddf)

In [12]:
bc.sql("select * from test").compute()

running on ucx://127.0.0.1:43864
running on ucx://127.0.0.1:60372


,A
0,0.0
1,1.0
2,2.0
3,3.0
4,4.0
...,...
2081,4167.0
2082,4168.0
2083,4169.0
2084,4170.0


In [ ]:
bc.sql("select test.A, test2.A from test full outer join test2 on test2.A = test.A ").compute()

running on ucx://127.0.0.1:43864
running on ucx://127.0.0.1:60372


In [29]:
bc.sql("select avg(a) from test group by b").compute()

Error found
fail: 
 org.apache.calcite.runtime.CalciteContextException: At line 1, column 34: Column 'b' not found in any table
Parsing Error


AttributeError: 'NoneType' object has no attribute 'compute'

In [7]:
data_dir = "/home/cjnolet/Downloads/data/tpch/"
bc.create_table("orders",data_dir + "orders*.parquet")
bc.create_table("nation",data_dir + "nation*.parquet")
bc.create_table("supplier",data_dir + "supplier*.parquet")
bc.create_table("region",data_dir + "region*.parquet")
bc.create_table("partsupp",data_dir + "partsupp*.parquet")
bc.create_table("part",data_dir + "part_*.parquet")
bc.create_table("lineitem",data_dir + "lineitem*.parquet")
bc.create_table("customer",data_dir + "customer*.parquet")


In [ ]:
print(bc.sql("""
select COALESCE(orders.o_orderkey, 100),
                    COALESCE(orders.o_totalprice, 0.01) from customer
                    full outer join orders
                    on customer.c_custkey = orders.o_custkey
                    where customer.c_nationkey = 3
                    and customer.c_custkey < 3
                    LIMIT 2
""").compute())
print(bc.sql("""
select o_orderkey, count(o_custkey) from orders group by o_orderkey
""").compute())

running on ucx://127.0.0.1:48212
running on ucx://127.0.0.1:50104
